In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import PIL
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from random import randint
from sklearn.model_selection import train_test_split


In [ ]:
root_dir = os.path.abspath('')
data_dir = os.path.join(root_dir, 'Images')
# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)
workers_test = pd.read_csv(os.path.join(data_dir,  'test_workers_dataset_normalized.csv'), sep=' ')

In [ ]:
from PIL import Image
class ImageRatingsDataset(Dataset):
    """Images dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images_frame = pd.read_csv(csv_file, sep=' ')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.images_frame)

    def __getitem__(self, idx):
        try:
            img_name = str(os.path.join(self.root_dir,str(self.images_frame.iloc[idx, 0])))
            im = Image.open(img_name).convert('RGB')
            if im.mode == 'P':
                im = im.convert('RGB')
            image = np.asarray(im)
            #image = io.imread(img_name+'.jpg', mode='RGB').convert('RGB')
            rating = self.images_frame.iloc[idx, 1]
            sample = {'image': image, 'rating': rating}

            if self.transform:
                sample = self.transform(sample)
            return sample            
        except Exception as e:
            pass
        

            
        

In [ ]:
import random
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image = transform.resize(image, (new_h, new_w))

        return {'image': image, 'rating': rating}

        
    

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]


        return {'image': image, 'rating': rating}

class RandomHorizontalFlip(object):
    def __init__(self, p):
        self.p = p
    
    def __call__(self, sample):
        
        image, rating = sample['image'], sample['rating']
        if random.random() < self.p:
            image = np.flip(image, 1)
            #image = io.imread(img_name+'.jpg', mode='RGB').convert('RGB')
        return {'image': image, 'rating': rating}

class Normalize(object):
    def __init__(self ):
        self.means=np.array([0.485, 0.456, 0.406]) 
        self.stds=np.array([0.229, 0.224, 0.225])
        
    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        im=image/255
        im[:,:,0]=(image[:,:,0] - 0.485)/ 0.229 
        im[:,:,1]=(image[:,:,1]  - self.means[1])/ self.stds[1]
        im[:,:,2]=(image[:,:,2]  - self.means[2])/ self.stds[2]
        image=im
        return {'image': image, 'rating': rating}

    
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
            
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).double(),
                'rating': torch.from_numpy(np.float64([rating])).double()}

In [ ]:
worker=workers_test['worker'].unique()[0]
num_images = workers_test[workers_test['worker'].isin([worker])].shape[0]
percent=100/num_images
images=workers_test[workers_test['worker'].isin([worker])][[' imagePair', ' score']]
train_dataframe, valid_dataframe = train_test_split(images, train_size=percent)
train_path="train_means_normalized" + worker +".csv"
test_path="test_means_normalized" + worker +".csv"

In [ ]:
train_dataframe.to_csv(train_path, sep=' ', index_label = False)
valid_dataframe.to_csv(test_path, sep=' ', index_label = False)

In [ ]:
output_size=(224,224)
transformed_dataset_train = ImageRatingsDataset(csv_file=train_path,root_dir='Images/',
                                           transform=transforms.Compose([Rescale(output_size=(256,256)),
                                                                         RandomHorizontalFlip(0.5),
                                                                         RandomCrop(output_size=output_size),
                                                                         Normalize(),
                                                                        ToTensor(),
                                           ]))
transformed_dataset_valid = ImageRatingsDataset(csv_file=test_path,root_dir='Images/',
                                           transform=transforms.Compose([Rescale(output_size=(224,224)),
                                                                         Normalize(),
                                                                        ToTensor(),
                                           ]))

In [ ]:
from torch.utils.data.dataloader import default_collate
bsize=12
def my_collate(batch):
    batch = list(filter (lambda x:x is not None, batch))
    return default_collate(batch)

dataloader_train = DataLoader(transformed_dataset_train, batch_size=bsize,
                        shuffle=True, num_workers=0,collate_fn=my_collate)
dataloader_valid = DataLoader(transformed_dataset_valid, batch_size=8,
                        shuffle=True, num_workers=0,collate_fn=my_collate)


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import copy
from torch import nn
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from random import randint

In [ ]:
from torchvision import models
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs,1))

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, num_classes, keep_probability, inputsize):
    
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(inputsize, 256)
        self.drop_prob = (1 - keep_probability)
        self.relu1= nn.PReLU()
        self.drop1 = nn.Dropout(self.drop_prob)
        self.bn1=nn.BatchNorm1d(256)
        self.fc2=nn.Linear(256,256)
        self.relu2=nn.PReLU()
        self.drop2 = nn.Dropout(p=self.drop_prob)
        self.bn2=nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Weight initialization reference: https://arxiv.org/abs/1502.01852
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        """
        Feed-forward pass.
        :param x: Input tensor
        : return: Output tensor
        """
        out=self.fc1(x)
        out =self.relu1(out)
        out=self.drop1(out)
        out=self.bn1(out)
        out=self.fc2(out)
        out=self.relu2(out)
        out=self.drop2(out)
        out=self.bn2(out)
        out=self.fc3(out)
        return out

In [ ]:
class convNet(nn.Module):
    #constructor
    def __init__(self,resnet,mynet):
        super(convNet, self).__init__()
        #defining layers in convnet
        self.resnet=resnet
        self.myNet=mynet
    def forward(self, x):
        x=self.resnet(x)
        x=self.myNet(x)
        return x

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.out_features
net1 = BaselineModel(1, 0.5,num_ftrs)
net2=convNet(resnet=model_ft, mynet=net1)

In [ ]:
net2

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
use_gpu = True
if use_gpu:
    cuda = torch.device('cuda:0')     # Default CUDA device
count=0
torch.cuda.set_device(cuda.index)

In [ ]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=5):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""

    if epoch % lr_decay_epoch == 0:
        #print('LR is set to {}'.format(lr))
        pass
    lr = init_lr * (0.9**(epoch // lr_decay_epoch))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
net_2 = (torch.load('fine_tuned_flickerAES_normalized_dropout_resnet18_customnetworkadamnormalized.pt'))

In [ ]:
class rsn18 (nn.Module):
    def __init__(self):
        super(rsn18, self).__init__()
        self.conv1 = models.resnet18().conv1
        self.bn1=models.resnet18().bn1
        self.relu=models.resnet18().relu
        self.maxpool=models.resnet18().maxpool
        self.adapter1_0=nn.Conv2d(64,64, kernel_size=1, padding=0)
        self.adapter1_1=nn.Conv2d(64,64, kernel_size=1, padding=0)
        self.adapter1_2=nn.Conv2d(64,64, kernel_size=1, padding=0)
        self.adapter1_3=nn.Conv2d(64,64, kernel_size=1, padding=0)
        
        
        self.adapter2_0=nn.Conv2d(64,128, kernel_size=1, padding=0, stride=2)
        self.adapter2_1=nn.Conv2d(128,128, kernel_size=1, padding=0)
        self.adapter2_2=nn.Conv2d(128,128, kernel_size=1, padding=0)
        self.adapter2_3=nn.Conv2d(128,128, kernel_size=1, padding=0)
        
        self.adapter3_0=nn.Conv2d(128,256, kernel_size=1, padding=0, stride=2)
        self.adapter3_1=nn.Conv2d(256,256, kernel_size=1, padding=0)
        self.adapter3_2=nn.Conv2d(256,256, kernel_size=1, padding=0)
        self.adapter3_3=nn.Conv2d(256,256, kernel_size=1, padding=0)
        
        self.adapter4_0=nn.Conv2d(256,512, kernel_size=1, padding=0, stride=2)
        self.adapter4_1=nn.Conv2d(512,512, kernel_size=1, padding=0)
        self.adapter4_2=nn.Conv2d(512,512, kernel_size=1, padding=0)
        self.adapter4_3=nn.Conv2d(512,512, kernel_size=1, padding=0)

        self.layer1=models.resnet18().layer1
        self.layer2=models.resnet18().layer2
        self.layer3=models.resnet18().layer3
        self.layer4=models.resnet18().layer4
        self.avgpool=models.resnet18().avgpool
        #nn.init.xavier_uniform_(self.adapter1.weight, gain=0.01)
        #nn.init.xavier_uniform_(self.adapter2.weight, gain=0.01)
        #nn.init.xavier_uniform_(self.adapter3.weight, gain=0.01)
        #nn.init.xavier_uniform_(self.adapter4.weight, gain=0.01)

        self.fc=models.resnet18().fc
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        residual=x
        x = self.layer1[0].conv1(x) + self.adapter1_0(x)
        x = self.layer1[0].bn1(x)
        x = self.layer1[0].relu(x)
        x = self.layer1[0].conv2(x) + self.adapter1_1(x)
        x = self.layer1[0].bn2(x)
        x = x + residual
        x = self.layer1[0].relu(x)
        
        residual=x
        x = self.layer1[1].conv1(x) + self.adapter1_2(x)
        x = self.layer1[1].bn1(x)
        x = self.layer1[1].relu(x)
        x = self.layer1[1].conv2(x) + self.adapter1_3(x)
        x = self.layer1[1].bn2(x)
        x = x + residual
        x = self.layer1[1].relu(x)
        
        
        residual=x
        x = self.layer2[0].conv1(x) + self.adapter2_0(x)
        x = self.layer2[0].bn1(x)
        x = self.layer2[0].relu(x)
        x = self.layer2[0].conv2(x) + self.adapter2_1(x)
        x = self.layer2[0].bn2(x)
        residual = self.layer2[0].downsample(residual)
        x = x + residual
        x = self.layer2[0].relu(x)
        
        residual=x
        x = self.layer2[1].conv1(x) + self.adapter2_2(x)
        x = self.layer2[1].bn1(x)
        x = self.layer2[1].relu(x)
        x = self.layer2[1].conv2(x) + self.adapter2_3(x)
        x = self.layer2[1].bn2(x)
        x = x + residual
        x = self.layer2[1].relu(x)
        
        
        residual=x
        x = self.layer3[0].conv1(x) + self.adapter3_0(x)
        x = self.layer3[0].bn1(x)
        x = self.layer3[0].relu(x)
        x = self.layer3[0].conv2(x) + self.adapter3_1(x)
        x = self.layer3[0].bn2(x)
        residual = self.layer3[0].downsample(residual)
        x = x + residual
        x = self.layer3[0].relu(x)
        
        residual=x
        x = self.layer3[1].conv1(x) + self.adapter3_2(x)
        x = self.layer3[1].bn1(x)
        x = self.layer3[1].relu(x)
        x = self.layer3[1].conv2(x) + self.adapter3_3(x)
        x = self.layer3[1].bn2(x)
        x = x + residual
        x = self.layer3[1].relu(x)
        
        
        residual=x
        x = self.layer4[0].conv1(x) + self.adapter4_0(x)
        x = self.layer4[0].bn1(x)
        x = self.layer4[0].relu(x)
        x = self.layer4[0].conv2(x) + self.adapter4_1(x)
        x = self.layer4[0].bn2(x)
        residual = self.layer4[0].downsample(residual)
        x = x + residual
        x = self.layer4[0].relu(x)
        
        residual=x
        x = self.layer4[1].conv1(x) + self.adapter4_2(x)
        x = self.layer4[1].bn1(x)
        x = self.layer4[1].relu(x)
        x = self.layer4[1].conv2(x) + self.adapter4_3(x)
        x = self.layer4[1].bn2(x)
        x = x + residual
        x = self.layer4[1].relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
rs=rsn18()

In [ ]:
rs.conv1.weight.data = net_2.resnet.conv1.weight.data
rs.bn1.weight.data = net_2.resnet.bn1.weight.data

rs.layer1[0].conv1.weight.data = net_2.resnet.layer1[0].conv1.weight.data
rs.layer1[0].bn1.weight.data = net_2.resnet.layer1[0].bn1.weight.data
rs.layer1[0].conv2.weight.data = net_2.resnet.layer1[0].conv2.weight.data
rs.layer1[0].bn2.weight.data = net_2.resnet.layer1[0].bn2.weight.data

rs.layer1[1].conv1.weight.data = net_2.resnet.layer1[1].conv1.weight.data
rs.layer1[1].bn1.weight.data = net_2.resnet.layer1[1].bn1.weight.data
rs.layer1[1].conv2.weight.data = net_2.resnet.layer1[1].conv2.weight.data
rs.layer1[1].bn2.weight.data = net_2.resnet.layer1[1].bn2.weight.data


rs.layer2[0].conv1.weight.data = net_2.resnet.layer2[0].conv1.weight.data
rs.layer2[0].bn1.weight.data = net_2.resnet.layer2[0].bn1.weight.data
rs.layer2[0].conv2.weight.data = net_2.resnet.layer2[0].conv2.weight.data
rs.layer2[0].bn2.weight.data = net_2.resnet.layer2[0].bn2.weight.data

rs.layer2[1].conv1.weight.data = net_2.resnet.layer2[1].conv1.weight.data
rs.layer2[1].bn1.weight.data = net_2.resnet.layer2[1].bn1.weight.data
rs.layer2[1].conv2.weight.data = net_2.resnet.layer2[1].conv2.weight.data
rs.layer2[1].bn2.weight.data = net_2.resnet.layer2[1].bn2.weight.data
rs.layer2[0].downsample[0].weight.data=net_2.resnet.layer2[0].downsample[0].weight.data
rs.layer2[0].downsample[1].weight.data=net_2.resnet.layer2[0].downsample[1].weight.data

rs.layer3[0].conv1.weight.data = net_2.resnet.layer3[0].conv1.weight.data
rs.layer3[0].bn1.weight.data = net_2.resnet.layer3[0].bn1.weight.data
rs.layer3[0].conv2.weight.data = net_2.resnet.layer3[0].conv2.weight.data
rs.layer3[0].bn2.weight.data = net_2.resnet.layer3[0].bn2.weight.data

rs.layer3[1].conv1.weight.data = net_2.resnet.layer3[1].conv1.weight.data
rs.layer3[1].bn1.weight.data = net_2.resnet.layer3[1].bn1.weight.data
rs.layer3[1].conv2.weight.data = net_2.resnet.layer3[1].conv2.weight.data
rs.layer3[1].bn2.weight.data = net_2.resnet.layer3[1].bn2.weight.data
rs.layer3[0].downsample[0].weight.data=net_2.resnet.layer3[0].downsample[0].weight.data
rs.layer3[0].downsample[1].weight.data=net_2.resnet.layer3[0].downsample[1].weight.data

rs.layer4[0].conv1.weight.data = net_2.resnet.layer4[0].conv1.weight.data
rs.layer4[0].bn1.weight.data = net_2.resnet.layer4[0].bn1.weight.data
rs.layer4[0].conv2.weight.data = net_2.resnet.layer4[0].conv2.weight.data
rs.layer4[0].bn2.weight.data = net_2.resnet.layer4[0].bn2.weight.data

rs.layer4[1].conv1.weight.data = net_2.resnet.layer4[1].conv1.weight.data
rs.layer4[1].bn1.weight.data = net_2.resnet.layer4[1].bn1.weight.data
rs.layer4[1].conv2.weight.data = net_2.resnet.layer4[1].conv2.weight.data
rs.layer4[1].bn2.weight.data = net_2.resnet.layer4[1].bn2.weight.data
rs.layer4[0].downsample[0].weight.data=net_2.resnet.layer4[0].downsample[0].weight.data
rs.layer4[0].downsample[1].weight.data=net_2.resnet.layer4[0].downsample[1].weight.data

rs.fc.weight.data=net_2.resnet.fc.weight.data

In [ ]:
class convNet2(nn.Module):
    #constructor
    def __init__(self,resnet,mynet):
        super(convNet2, self).__init__()
        #defining layers in convnet
        self.rsn=resnet
        self.myNet=mynet
    def forward(self, x):
        x=self.rsn(x)
        x=self.myNet(x)
        return x

In [ ]:
cnvnet2=convNet2(resnet=rs, mynet=net1)

In [ ]:
cnvnet2.myNet.bn1.weight.data=net_2.myNet.bn1.weight.data
cnvnet2.myNet.fc2.weight.data=net_2.myNet.fc2.weight.data
cnvnet2.myNet.bn2.weight.data=net_2.myNet.bn2.weight.data
cnvnet2.myNet.fc3.weight.data=net_2.myNet.fc3.weight.data

In [ ]:
torch.save(cnvnet2, 'adaptersnetwork1.pt')


In [ ]:
cnvnet2

In [ ]:
from scipy.stats import spearmanr
def computeSpearman(dataloader_valid, model):
    ratings = []
    predictions=[]
    #device = cuda
    #criterion = nn.MSELoss()
    #criterion.cuda()
    with torch.no_grad():
        cum_loss=0
        for batch_idx, data in enumerate(dataloader_valid):
            inputs = data['image']
            labels=data['rating']
            if use_gpu:
                try:
                    inputs, labels = Variable(inputs.float().cuda()), Variable(labels.float().cuda())
                except:
                    print(inputs,labels)
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            outputs = model(inputs)
            ratings.append(labels.float())
            predictions.append(outputs.float())
    
    ratings_i=[(list(np.float_([j for j in i]))) for i in ratings]
    predictions_i=[(list(np.float_([j for j in i]))) for i in predictions]
    ratings_i=np.vstack(ratings)
    predictions_i=np.vstack(predictions)
    from scipy.stats import spearmanr
    return spearmanr(ratings_i, predictions_i)

In [ ]:
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def train_model(model, criterion, optimizer1, optimizer2, lr_scheduler,dataloader_train,dataloader_valid,  num_epochs=100):
    since = time.time()
    train_loss_average=[]
    test_loss=[]
    spearman_test=[]
    best_model = model
    best_loss = 100
    best_spearman=0
    use_gpu=True
    model_1=model
    criterion.cuda()
    for epoch in range(num_epochs):
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val', 'train']:
            if phase == 'train':
                model=model_1
                mode='train'
                optimizer1 = lr_scheduler(optimizer1, epoch)
                optimizer2 = lr_scheduler(optimizer2, epoch)
                model.train()  # Set model to training mode
                dataloader=dataloader_train
            elif phase=='val' and epoch==0:
                net_2 = (torch.load('fine_tuned_flickerAES_normalized_dropout_resnet18_customnetworkadamnormalized.pt'))
                model=net_2
                model.eval()
                mode='val'
                dataloader=dataloader_valid
            else:
                model=model_1
                model.eval()
                mode='val'
                dataloader=dataloader_valid

            running_loss = 0.0
            model.cuda()

            counter=0
            # Iterate over data.
            for batch_idx, data in enumerate(dataloader):
                inputs = data['image']
                labels=data['rating']
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()), Variable(labels.float().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                # wrap them in Variable

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer1.zero_grad()
                optimizer2.zero_grad()
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                #print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                #if counter%200==0:
                    #print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    #print('loss backward')
                    loss.backward()
                    #print('done loss backward')
                    optimizer1.step()
                    optimizer2.step()

                    #print('done optim')
                # print evaluation statistics
                try:
                    running_loss += loss.data[0]
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            #print('trying epoch loss')
            epoch_loss = running_loss / len(dataloader)
            #print('{} Loss: {:.4f} '.format(
            #    phase, epoch_loss))
            if phase == 'train':
                train_loss_average.append(epoch_loss)


            # deep copy the model
            if phase == 'val':
                test_loss.append(epoch_loss)
                sp=computeSpearman(dataloader, model)[0]
                spearman_test.append(sp)

                if epoch_loss < best_loss:
                    best_loss=epoch_loss
                    #print('new best loss = ',epoch_loss)
                
                if sp>best_spearman:
                    best_spearman=sp
                    best_model = copy.deepcopy(model)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    print('returning and looping back')
    return best_model.cuda(), train_loss_average, np.asarray(test_loss), np.asarray(spearman_test)

In [ ]:
from torchvision import models
from torch import optim
import torch.nn.functional as F

model_ft=cnvnet2


#device = cuda
criterion = nn.MSELoss()
criterion.cuda()
model_ft.cuda()
for name, m in model_ft.named_modules():
        if isinstance(m, nn.Conv2d) and (m.kernel_size[0]==3 or m.kernel_size[0]==7):
            m.weight.requires_grad = False
model_ft.rsn.fc.weight.requires_grad=False
model_ft.rsn.layer2[0].downsample[0].weight.requires_grad=False
model_ft.rsn.layer3[0].downsample[0].weight.requires_grad=False
model_ft.rsn.layer4[0].downsample[0].weight.requires_grad=False



optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001, weight_decay=0.001)

In [ ]:
def ComputeIncreaseSP(sp):
    return max(sp) - sp[0]

def ComputeDecreaseMSE(MSEerrors):
    return min(MSEerrors)-MSEerrors[0]

In [ ]:
epochs=100
stats=[]
for worker_idx in range(0,37):
    resu= []
    for i in range(0,3):
        worker=workers_test['worker'].unique()[worker_idx]
        print(worker, i)
        num_images = workers_test[workers_test['worker'].isin([worker])].shape[0]
        percent=100/num_images
        images=workers_test[workers_test['worker'].isin([worker])][[' imagePair', ' score']]
        train_dataframe, valid_dataframe = train_test_split(images, train_size=percent)
        train_path="train_means_normalized" + worker +".csv"
        test_path="test_means_normalized" + worker +".csv"
        train_dataframe.to_csv(train_path, sep=' ', index_label = False)
        valid_dataframe.to_csv(test_path, sep=' ', index_label = False)

        output_size=(224,224)
        transformed_dataset_train = ImageRatingsDataset(csv_file=train_path,root_dir='Images/',
                                               transform=transforms.Compose([Rescale(output_size=(256,256)),
                                                                             RandomHorizontalFlip(0.5),
                                                                             RandomCrop(output_size),
                                                                             Normalize(),
                                                                            ToTensor(),
                                               ]))
        transformed_dataset_valid = ImageRatingsDataset(csv_file=test_path,root_dir='Images/',
                                               transform=transforms.Compose([Rescale(output_size=(224,224)),
                                                                             Normalize(),
                                                                            ToTensor(),
                                               ]))

        from torch.utils.data.dataloader import default_collate
        bsize=30
        def my_collate(batch):
            batch = list(filter (lambda x:x is not None, batch))
            return default_collate(batch)

        dataloader_train = DataLoader(transformed_dataset_train, batch_size=bsize,
                                shuffle=True, num_workers=0,collate_fn=my_collate)
        dataloader_valid = DataLoader(transformed_dataset_valid, batch_size=20,
                                shuffle=True, num_workers=0,collate_fn=my_collate)


        cnvnet2 = (torch.load('adaptersnetwork1.pt'))
        model_ft=cnvnet2
        for name, m in model_ft.named_modules():
            if isinstance(m, nn.Conv2d) and (m.kernel_size[0]==3 or m.kernel_size[0]==7):
                m.weight.requires_grad = False
        model_ft.rsn.fc.weight.requires_grad=False
        model_ft.rsn.layer2[0].downsample[0].weight.requires_grad=False
        model_ft.rsn.layer3[0].downsample[0].weight.requires_grad=False
        model_ft.rsn.layer4[0].downsample[0].weight.requires_grad=False

        #device = cuda
        criterion = nn.MSELoss()
        criterion.cuda()
        model_ft.cuda()
        optimizer1 = optim.Adam(filter(lambda p: p.requires_grad, model_ft.rsn.parameters()),  lr=0.001, weight_decay=0.01)
        optimizer2 = optim.Adam(filter(lambda p: p.requires_grad, model_ft.myNet.parameters()),  lr=0.001, weight_decay=0)

        iteration=0
        network_path= worker + 'network_0 K100 adapters it' + str(i) + '.pt'
        # Run the functions and save the best model in the function model_ft.
        model_ft, train_loss, test_loss, spearman = train_model(model_ft, criterion, optimizer1, optimizer2, exp_lr_scheduler,dataloader_train,dataloader_valid,
                               num_epochs=epochs)
 

        #torch.save(model_ft, network_path)
        results=worker, ComputeDecreaseMSE(test_loss), ComputeIncreaseSP(spearman), np.argmin(test_loss), np.argmax(spearman)
        resu.append(results)
        
        
    resupd=pd.DataFrame(resu)
    resupd.columns=['Worker', 'DecreaseMSE', 'IncreaseSpearman', 'ItMSE', 'ItSpearman']
    path=worker+'k100 adapters.csv'
    resupd.to_csv(path, sep=' ', index_label = False)
    number=images[' imagePair'].unique().shape[0]
    stat=resupd['Worker'][0],number, np.mean(resupd['DecreaseMSE']), np.std(resupd['DecreaseMSE']), np.mean(resupd['IncreaseSpearman']), np.std(resupd['IncreaseSpearman']), np.mean(resupd['ItMSE']), np.std(resupd['ItMSE']),np.mean(resupd['ItSpearman']), np.std(resupd['ItSpearman']),
    stats.append(stat)
stats=pd.DataFrame(stats)
stats.to_csv('100 epochs k100 adaptersfinetuning01 WD01.csv', sep=' ', index_label=False)

In [ ]:
def ComputeIncreaseSP(sp):
    return max(sp) - sp[0]

def ComputeDecreaseMSE(MSEerrors):
    return min(MSEerrors)-MSEerrors[0]

In [ ]:
for name, m in model_ft.named_modules():
    if (isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear) or isinstance(m, nn.BatchNorm1d) or isinstance(m, nn.BatchNorm2d)):
        print(name, str(m.weight.requires_grad))
    

In [ ]:
%matplotlib inline
from graphviz import Digraph
import re
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Variable
import torchvision.models as models


def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

inputs = torch.randn(2,3,224,224)
y = model_ft(Variable(inputs.cuda()))
print(y)

g = make_dot(y, model_ft.state_dict())
g

In [ ]:
g.render()